# Getting Started - Lab 01 - Vectara Query API

We'll now explore the Vectara Query API.

This notebook will use our "lab" authentication profile, if you haven't set this up, please [Setup Authentication](./00_setup_authentication.ipynb).

In [ ]:
from vectara.factory import Factory
from vectara.managers import CreateCorpusRequest
from getting_started_util import GettingStartedUtil

util = GettingStartedUtil()
logger = util.logger
client = Factory(profile="lab").build()


## Setup Corpus and Data
Before we can run queries, we setup a corpus and some data. We'll examine this in more depth in the following notebooks.

In [ ]:
corpus_key = util.setup_01(client)

# Query Types
Vectara supports three query interfaces, in addition to chat which will be covered seperately.

1. Simple Single Corpus Query
2. Advanced Single Corpus Query
3. Multiple Corpora Query 

We'll now examine how you can invoke each of these with the SDK.

## Single Corpus Query
The following code performs a query against a single corpus using the convenience method which invokes [Simple Single Corpus Query REST API](https://docs.vectara.com/docs/rest-api/search-corpus).

In [ ]:
from vectara.types import GenerationParameters 
from vectara.utils import render_markdown
from IPython.display import display, Markdown
import json

query = "What is the Wager about?"

query_response = client.corpora.query(corpus_key, query=query)
display(Markdown(render_markdown(query, query_response)))

## Advanced Single Corpus Query
The following code performs a query against a single corpus using the advanced method which invokes [Advanced Single Corpus Query REST API](https://docs.vectara.com/docs/rest-api/query-corpus).

In [ ]:
from vectara.corpora.types import SearchCorpusParameters

generation = GenerationParameters.parse_obj({
    "generation_preset_name": "vectara-summary-ext-v1.3.0",
    "max_used_search_results": 5,
    "max_response_characters": 300,
    "response_language": "auto"
})

search_corpus = SearchCorpusParameters.parse_obj({
    "lexical_interpolation": 0.025,
    "semantics": "default",
    "offset": 0,
    "limit": 10,
    "reranker": {
        "type": "customer_reranker",
        "reranker_id": "rnk_272725719"
    },
    "context_configuration": {
        "characters_before": 30,
        "characters_after": 30,
        "start_tag": "<b>",
        "end_tag": "</b>"
    },
})

query_response = client.corpora.query(corpus_key, query=query, search=search_corpus, generation=generation)
display(Markdown(render_markdown(query, query_response, show_search_results=True)))

# Multiple Corpora Query (Not working, SDK refactor underway)
We will now show how you can invoke the multiple-corpora query for advanced use cases which invoke [Multiple Corpora Query REST API](https://docs.vectara.com/docs/rest-api/query).

Please note the two similarly named types:
* `vectara.queries.SearchCorpusParameters` - Used above to search a single corpus
* `vectara.types.SearchCorporaParameters` - Used hear to search multiple corpora

Also observe the slight difference in methods in `vectara.queries.QueryClient`:
* `QueryClient#query_corpus` - Used above to search a single corpus
* `QueryClient#query` - Used here to perform a multi-corpora query

In [ ]:
from vectara.types import SearchCorporaParameters

search = SearchCorporaParameters.parse_obj({
    "corpora": [{"corpus_key": corpus_key}] # This can be multiple corpora
})


query_response = client.corpora.query(corpus_key, query=query, search=search, generation=generation)
display(Markdown(render_markdown(query, query_response, show_search_results=True)))